In [2]:
%run metricas.ipynb 

mkdir: no se puede crear el directorio «build»: El archivo ya existe
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/awolfsdorf/.pyenv/versions/mettp3/bin/python (found version "3.7.5") 
-- Found PythonLibs: /home/awolfsdorf/.pyenv/versions/3.7.5/lib/libpython3.7m.a
-- pybind11 v2.2.4
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/awolfsdorf/Dropbox

In [75]:
df_original = pd.read_csv('../data/train.csv')
df = df_original.copy()
#df

SEGMENTACIÓN NUMÉRICA: \
la categoria seleccionada tiene que contener numeros, no palabras \
    n = cantidad de particiones \
    df es la matriz con toda la informacion \
    palabra es la columna por la cual segmentaremos

In [76]:
def segXcatNum(df, n , palabra):
    
    df = df[df['ciudad']=='Mérida'].dropna() #TODO sin esto no anda, averiguar por que
    
    tam = df.shape[0]
    linear_regressor = metnum.LinearRegression()
    
    
    df.sort_values("precio") #ordena de menor a mayor
    
    #tomo la columna de PALABRA y la de precios
    categoria = df[palabra].values
    precios = df['precio'].values
    vector = np.array([])
    for i in range(n):
        
        #tomo n bloques de longitud tam/n
        bloqueCategoria = categoria[int(i * tam/n) : int((i+1) * (tam/n))]
        bloquePrecios = precios[int(i * tam/n) : int((i+1) * tam/n)]
        
        #reshapeo
        bloqueCategoria = bloqueCategoria.reshape(len(bloqueCategoria),1)
        bloquePrecios = bloquePrecios.reshape(len(bloquePrecios),1)
        
        #fit y prediccion 
        linear_regressor.fit(bloqueCategoria, bloquePrecios)
        lin = linear_regressor.predict(bloqueCategoria)
        lin = lin.reshape(lin.shape[0])
        #reshapeo para poder meterlo en un array con el formato necesario
        vector = np.append(vector, lin)
        
    #df['prediction'] = vector ---- sirve para plotear cosas mas facil
    return vector





In [77]:
segmentado = segXcatNum(df, 5, 'metroscubiertos')
merida = df[df['ciudad']=='Mérida'].dropna() #TODO sin esto no anda, averiguar por que
original = merida['precio'].values
original = original.reshape(len(original))
print(original.shape, segmentado.shape)
print(
    "Correlacion: \n ",
coefCorrel(segmentado, original), "\n",
    "Determinacion: \n",
    coefDet(segmentado, original),  "\n",
    "RMSE: \n",
RMSE(segmentado, original), "\n",
    "RMSLE: \n", 
RMSLE(segmentado, original),

)

(518,) (518,)
Correlacion: 
  0.7421713193836197 
 Determinacion: 
 0.5508182673156228 
 RMSE: 
 676017.2546645574 
 RMSLE: 
 0.43092419953725103


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          240000 non-null  int64  
 1   titulo                      234613 non-null  object 
 2   descripcion                 238381 non-null  object 
 3   tipodepropiedad             239954 non-null  object 
 4   direccion                   186928 non-null  object 
 5   ciudad                      239628 non-null  object 
 6   provincia                   239845 non-null  object 
 7   antiguedad                  196445 non-null  float64
 8   habitaciones                217529 non-null  float64
 9   garages                     202235 non-null  float64
 10  banos                       213779 non-null  float64
 11  metroscubiertos             222600 non-null  float64
 12  metrostotales               188533 non-null  float64
 13  idzona        

In [79]:
# Ideas de 

# Ideas de modelos
- Centros comerciales cercanos según antiguedad => centroscomercialescercanos = a x antiguedad + b
- Feature engeniere -> si esta cerca de una gran ciudad sube el precio.  -> para hacer Segmentacion
-                   -> Casa de gente con guita: gimnasio + piscina + tipopropiedad(casa, quinta, villa, alguno mas...) -> para hacer Segmentacion
-                   -> Ponerle pesos a algunas variables categoricas y sumarlas -> 
- Usar PCA para extraer componentes principales

In [80]:
df['tipodepropiedad'].value_counts()

Casa                             141717
Apartamento                       57341
Casa en condominio                19297
Terreno                            9945
Local Comercial                    3055
Oficina comercial                  1741
Bodega comercial                   1406
Edificio                           1396
Terreno comercial                  1326
Casa uso de suelo                   708
Quinta Vacacional                   395
Duplex                              343
Villa                               340
Inmuebles productivos urbanos       200
Rancho                              170
Local en centro comercial           165
Departamento Compartido             141
Otros                               134
Nave industrial                      76
Terreno industrial                   31
Huerta                               20
Lote                                  5
Garage                                1
Hospedaje                             1
Name: tipodepropiedad, dtype: int64

In [81]:
df[df['tipodepropiedad'] == 'Villa'].sort_values('precio')['precio']

95426       320000.0
7534        350000.0
8430        420000.0
234074      440000.0
149492      450000.0
             ...    
224173     9000000.0
130327     9000000.0
219406    10000000.0
60297     12000000.0
89786     12206700.0
Name: precio, Length: 340, dtype: float64

In [82]:
df[df['tipodepropiedad'] == 'Casa'].sort_values('precio')['precio']

184407      310000.0
30437       310000.0
189220      310000.0
133735      310000.0
213712      310000.0
             ...    
215495    12500000.0
92689     12500000.0
173245    12500000.0
232457    12515000.0
209858    12521000.0
Name: precio, Length: 141717, dtype: float64

In [83]:
def regresionPorSegmento(df, segment_column, features):
    predictions_df = pd.DataFrame()

    for segment in df[segment_column].unique():
        linear_regressor = metnum.LinearRegression()
        # Aquellos datos que no contengan la columna, no son tomados en cuenta
        df_segment = df[df[segment_column] == segment]

        A = df_segment[features].values
        precios_reales = df_segment['precio'].values

        linear_regressor.fit(A, precios_reales)
        precios_aproximados = linear_regressor.predict(A)
        precios_aproximados = precios_aproximados.reshape(precios_aproximados.shape[0])

        predictions_df[segment] = (linear_regressor, np.array(list(zip(precios_aproximados, precios_reales))))

    return predictions_df

In [84]:
# Modelo fruta
df = df_original.copy()
segment = 'tipodepropiedad'
features = ['metroscubiertos']#, 'metrostotales']
df = df[df[segment].notna() & df[features[0]].notna()]
df = df.sort_values(segment)
# df = df[df[segment] == 'Apartamento']

predictions = regresionPorSegmento(df, segment, features)


In [74]:
print(len(predictions['Apartamento'][1]))
print(len(df))
#print(df[segment].unique())
print(predictions['Apartamento'][1][:,1])

56518
222568
[2273000. 3000000.  980000. ... 3200000. 2200000. 1595000.]


,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
111255,73260,tres lagos laguna de mayran,lindo dpto de 60 mts con 2 recs y 2 baños y 2 ...,Apartamento,laguna de mayran,Miguel Hidalgo,Distrito Federal,5.0,2.0,2.0,...,275308.0,19.442137,-99.185902,2016-12-02 00:00:00,1.0,1.0,1.0,1.0,1.0,3000000.0
111248,823,departamento nuevo centro de poblacion,"departamento en nuevo centro de población, 3 r...",Apartamento,NaN,Acapulco de Juárez,Guerrero,NaN,3.0,1.0,...,38015.0,0.000000,0.000000,2016-12-29 00:00:00,0.0,0.0,0.0,0.0,0.0,980000.0
111246,25760,departamento en venta colinas del sur $1 pesos.,departamento en venta en la colonia colinas de...,Apartamento,"Calle Pregonero, entre Serenata y Av. Sta. Lucia",Alvaro Obregón,Distrito Federal,20.0,2.0,1.0,...,23263.0,19.346523,-99.191290,2016-11-06 00:00:00,0.0,0.0,0.0,0.0,0.0,1060666.0
111242,161878,departamento en venta en zona valle oriente,nocnok id: mx16-bw9256. especialistas en dep...,Apartamento,NaN,San Pedro Garza García,Nuevo León,NaN,3.0,NaN,...,NaN,NaN,NaN,2016-10-24 00:00:00,0.0,0.0,0.0,0.0,0.0,3600000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13594,18820,la colmena,"<p>hermosa casa campestre, en una área de 7,50...",Villa,NaN,Hermosillo,Sonora,NaN,3.0,NaN,...,125325.0,NaN,NaN,2015-01-13 00:00:00,0.0,0.0,0.0,0.0,0.0,3500000.0
69537,70767,la villa que atiende todas tus necesidades de ...,"villa con cocina equipada en pre venta, con op...",Villa,Selvamar,Solidaridad,Quintana Roo,NaN,3.0,2.0,...,87016.0,20.681321,-87.035926,2016-08-19 00:00:00,0.0,0.0,1.0,1.0,1.0,3960000.0
130047,258001,"marina vallarta, puerto vallarta jalisco 3 rec...","seguridad, ubicación, exclusividad, \ntodo eso...",Villa,puerto vallarta jalisco,Puerto Vallarta,Jalisco,4.0,3.0,2.0,...,50951.0,20.673634,-105.252577,2014-12-10 00:00:00,0.0,0.0,1.0,1.0,1.0,3800000.0
127889,294159,quinta campestre en colinas del vergel,<p>preciosa finca campestre en colonia privada...,Villa,NaN,Allende,Nuevo León,NaN,3.0,NaN,...,NaN,NaN,NaN,2014-08-29 00:00:00,0.0,0.0,0.0,0.0,0.0,2500000.0


In [107]:
import seaborn as sns

# sns.scatterplot(data=df, x='metroscubiertos', y='precio')

df_new = df.copy()
df_new["precio_aproximado"] = np.nan
for s in df_new[segment].unique():
    df_new[df_new[segment]==s]['precio_aproximado'] = 0

df_new["precio_aproximado"] 
# g = sns.FacetGrid(df_new, col=segment)
# g.map(plt.scatter, "");



# for s in df[segment].unique():
    
#     df_s = df[df[segment] == s]
#     df_s['precios_aproximados'] = predictions.loc[1, s][:,0]

#     sns.lineplot(data=df_s, x='metroscubiertos', y='precios_aproximados', color='red')


/home/awolfsdorf/.pyenv/versions/3.7.5/envs/mettp3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0        NaN
111255   NaN
111248   NaN
111246   NaN
111242   NaN
          ..
13594    NaN
69537    NaN
130047   NaN
127889   NaN
66011    NaN
Name: precio_aproximado, Length: 222568, dtype: float64